In [ ]:
''' Creando entorno virtual
python -m venv env
.\env\Scripts\activate  
'''

In [4]:
''' Importando Librerias'''
import pandas as pd
import seaborn as sb
import re


In [ ]:
# pip install openpyxl
# pip install seaborn  
# pip freeze > requirements.txt

# Cargar de Datos

In [471]:
'''Se carga las bases de datos'''
df_2022= pd.read_excel('../Datos/bookings-report-11944-20230105191605_2022.xlsx',engine="openpyxl")
df_2021= pd.read_excel('../Datos/bookings-report-11944-20230110192000_2021.xlsx',engine="openpyxl")
df_2020= pd.read_csv('../Datos/bookings-report-11944-20230105195054_2020.csv')
df_2019= pd.read_csv('../Datos/bookings-report-11944-20230105195220_2019.csv')
df_2018= pd.read_csv('../Datos/bookings-report-11944-20230105195327_2018.csv')
df_2017= pd.read_csv('../Datos/bookings-report-11944-20230105195426_2017.csv')
df_2016= pd.read_csv('../Datos/bookings-report-11944-20230105195530_2016.csv')
df_2015= pd.read_csv('../Datos/bookings-report-11944-20230105195614_2015.csv')
df_2014= pd.read_csv('../Datos/bookings-report-11944-20230105195714_2014.csv')


c:\Users\yoe_1\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\yoe_1\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [472]:
'''Uniendo las bases de datos'''
df_merge = pd.concat([df_2014, df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021,df_2022], ignore_index=True, sort=True)

'''Validando datos unicos'''
df_merge[df_merge.duplicated()]


,Adults,Arrival Airline,Arrival Drop-off,Arrival Flight No,Arrival Time,Barcodes,Booking Date,Booking Reference,Booking Source,Children,...,Product Code,Product Name,Seniors,Status,Supplier Confirmation Number,Tour Grade Code,Tour Grade Title,Tour Language,Travel Date,Youths


In [473]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Adults                        385 non-null    object 
 1   Arrival Airline               0 non-null      object 
 2   Arrival Drop-off              0 non-null      object 
 3   Arrival Flight No             0 non-null      object 
 4   Arrival Time                  0 non-null      object 
 5   Barcodes                      0 non-null      object 
 6   Booking Date                  385 non-null    object 
 7   Booking Reference             385 non-null    object 
 8   Booking Source                385 non-null    object 
 9   Children                      385 non-null    object 
 10  Currency                      385 non-null    object 
 11  Departure Airline             0 non-null      object 
 12  Departure Date                0 non-null      object 
 13  Depar

In [474]:
'''Seleccion de columnas relevantes'''
df_merge=df_merge.dropna(axis=1,how='all')
df_merge = df_merge.drop(['Infants','Seniors','Number of Passengers','Children','Youths','Lead traveler Contact Info'],axis=1)

'''Fijando Formato Fecha'''
df_merge['Booking Date'] = pd.to_datetime(df_merge['Booking Date'])


Se completan valores Nulos

In [475]:
''' Se completan los datos faltantes del Idioma con English'''
df_merge['Tour Language'].value_counts()
df_merge['Tour Language']=df_merge['Tour Language'].fillna('English')

'''Se estandariza la vatiable Mobil'''
df_merge['Mobile/Internet Channel']=df_merge['Mobile/Internet Channel'].replace('Mobile Apps','Mobile')

'''Completando algunos datos Vacios de Tour Grade Title'''# (Se reviso dato x dato)
df_merge['Tour Grade Title'].loc[30]='Kayak Jungle Tour'
df_merge['Tour Grade Title'].loc[31]='Kayak Jungle Tour'
df_merge['Tour Grade Title'].loc[44]='Class IV-III Half Day Morning'
df_merge['Tour Grade Title'].loc[69]='Class IV-III Full Day'
df_merge['Tour Grade Title'].loc[78]='Class IV-III Full Day'


'''Estandarizamos variable Hotel Pickup (1: Pichup - 0: No Pickup)'''
df_merge['Hotel Pickup']=df_merge['Hotel Pickup'].fillna(0, inplace = False)

def pickup(e): 
    if e != 0 : 
        return 1
    else: return 0

df_merge['Hotel Pickup']=df_merge['Hotel Pickup'].apply(pickup)

C:\Users\yoe_1\AppData\Local\Temp\ipykernel_11720\859254709.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['Tour Grade Title'].loc[30]='Kayak Jungle Tour'
C:\Users\yoe_1\AppData\Local\Temp\ipykernel_11720\859254709.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['Tour Grade Title'].loc[31]='Kayak Jungle Tour'
C:\Users\yoe_1\AppData\Local\Temp\ipykernel_11720\859254709.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

## Categorización del dato
Estas columnas corresponden al tipo de dato numérico

| categorical 	     | numerical 	            |
|-----------	     |-----------	            |
|Status - nominal    | Adults  - discrete     	|
|                    | Net Price - continuous   |

In [476]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Adults                   385 non-null    object        
 1   Booking Date             385 non-null    datetime64[ns]
 2   Booking Reference        385 non-null    object        
 3   Booking Source           385 non-null    object        
 4   Currency                 385 non-null    object        
 5   Email                    385 non-null    object        
 6   Hotel Pickup             385 non-null    int64         
 7   Lead traveler Name       385 non-null    object        
 8   Mobile/Internet Channel  385 non-null    object        
 9   Net Price                385 non-null    float64       
 10  Product Code             385 non-null    object        
 11  Product Name             385 non-null    object        
 12  Status                   385 non-nul

Actividad Turistica Según su Codigo de Producto
--
* 11944P1 : 'Best Whitewater Rafting Sarapiqui River, Costa Rica, Class II-III'

* 11944P2 : 'Best Whitewater Rafting Sarapiqui River, Costa Rica, Class III-IV'

* 11944P5 : 'Kayak Jungle Tour - Sarapiqui River - Costa Rica'

* 11944P12: 'Sarapiqui River Costa Rica White Water Rafting tour from Arenal, La Fortuna'

* 11944P11: 'Pacuare River White Water Rafting Adventure Tour, Costa Rica'

* 11944P3 : 'Safari Float in The Sarapiqui River'

* 11944P6 : 'White water Rafting Sarapiqui River'

* 11944P4 : 'Overnight White Water Rafting Sarapiqui'

In [477]:
'''Se crea la Columna Actividad y elimina las columnas Product Code y Product Name Code '''
def prod_cod(codigo) :
    if codigo == '11944P1':
       return 'Rafting,Class II-III'
    elif codigo == '11944P2':
        return 'Rafting, Class III-IV'
    elif codigo == '11944P5':
        return 'Kayak Jungle Tour'
    elif codigo == '11944P12':
        return 'Rafting tour, La Fortuna(Arenal)'
    elif codigo == '11944P11':
        return 'Rafting Tour, Pacuare River'
    elif codigo == '11944P3':
        return 'Safari Float in The Sarapiqui River'
    elif codigo == '11944P6':
        return 'White water Rafting Sarapiqui River'
    elif codigo == '11944P4':
        return 'Overnight White Water Rafting Sarapiqui'

df_merge['Actividad'] = df_merge['Product Code'].apply(prod_cod)
df_merge= df_merge.drop(['Product Code','Product Name'],axis=1)


In [478]:
'''Se eliminan espacios en blanco dentro del dataset'''
df_merge['Tour Grade Title'].str.strip()

'''Se standarizan la columna Horario'''
def horario(hora) : 
    if hora == 'TG1~07:00' or hora == 'TG1~06:00' or hora == 'TG1~07:30' or hora == 'TG1~08:00' or hora == 'TG1~09:00' or hora == 'TG2~09:00' or (hora.rfind('Full Day')>0) or (hora.rfind('07:00')>0) or (hora.rfind('08:00')>0) or (hora.rfind('07:30')>0) or  hora=='Vormittagstour' or (hora.rfind('mañana')>0) or  hora=='Kayak Jungle Tour' or (hora.rfind('06:00')>0)  :
       return 'De 6 AM a 9 AM'
    elif hora == 'TG1~10:00' or hora == 'TG4~10:00' or  hora == 'TG2~11:00' or hora == 'TG1~11:00' or hora == 'TG3~11:00' or hora == 'TG4~11:00' or (hora.rfind('Morning')>0) or hora=='Morning Tour'or (hora.rfind('11:00')>0) or (hora.rfind('10:00')>0) or (hora.rfind('09:00')>0) or (hora.rfind('Midday')>0) or  hora=='Mid-day Tour' or (hora.rfind('journée')>0) or  hora=='10:00'   :
        return 'De 10 AM a 12 PM'
    elif hora == 'TG2~13:00' or hora == 'TG1~13:00' or hora == 'TG1~15:00' or hora == 'TG2~15:00' or hora == 'TG2~14:00' or hora == 'TG2~14:30'   or hora == 'TG2~16:00'  or (hora.rfind('Afternoon')>0) or hora=='Afternoon Tour' or (hora.rfind('14:00')>0) or (hora.rfind('15:00')>0) or (hora.rfind('13:00')>0) or (hora.rfind('16:00')>0) or (hora.rfind('tarde')>0) :
        return 'De 1 PM a 3 PM'

df_merge['Horario'] = df_merge['Tour Grade Code'].apply(horario)
df_merge['Horario'] = df_merge['Tour Grade Title'].apply(horario)
# df_travel= df_travel.drop(['Tour Grade Code','Tour Grade Title'],axis=1)


In [479]:
''' Convertiendo valores a Dummies'''
df_merge_dum = pd.get_dummies(df_merge[['Status', 'Tour Language','Mobile/Internet Channel','Booking Source']],columns=['Status', 'Tour Language','Mobile/Internet Channel','Booking Source'])
df_merge=pd.concat([df_merge, df_merge_dum], axis=1)

In [480]:
''' Guardar todos los archivo '''
df_merge.to_csv('./Transformados/df_merge.csv',index=False)

---